#### Decomposable functions

Recall the concept of `gradient descent` is to, at iteration $k$, minimize the quadratic approximation of $f$ at $x^k$

$$x^{k+1}=\arg \min_xf(x^{k})+\nabla f(x^{k})^T(x-x^{k})+\frac{1}{2t^k}\|x-x^{k}\|_2^2$$

We take derivative and set it to zero

$$0+\nabla f(x^{k})+\frac{1}{t^k}(x-x^{k})=0$$

and we get the gradient descent equation

$$x^{k+1}=x^{k}-t^k \nabla f(x^{k})$$

The main motivation of proximal gradient method is that, when $f$ is not differentiable, rather than replacing the whole thing using subgradient, which we have seen is slow, we would like to keep some portion of this quadratic approximation

In particular, we look at function

$$f(x)=g(x)+h(x)$$

where $g$ is differentiable while $h$ is not

and we apply the quadratic approximation to $g$

$$\begin{align*}
x^{k+1} &  = \arg \min_z g(x^k)+\nabla g(x^k)^T(z-x^k)+\frac{1}{2t}\|z-x^k\|_2^2+h(z)\\
& g(x^k), \nabla g(x^k) \text{ as individual term has no effect on minimization over z}\\
 & = \arg \min_z \frac{1}{2} t\|\nabla g(x^k)\|_2^2+\nabla g(x^k)^T(z-x^k)+\frac{1}{2t}\|z-x^k\|_2^2+h(z)\\
&=\arg\min_z\frac{1}{2t}\|z-\left(x^k-t\nabla g(x^k)\right)\|_2^2+h(z)
\end{align*}$$

That is, we try to find a $z$ such that

* it stays close to gradient update of $g$ by minimizing $\|z-\left(x^k-t\nabla g(x^k)\right)\|_2^2$
* it makes $h$ small by minimizing $h(z)$

#### Proximal operator

More formally, we define proximal operator as

$$\text{prox}_{th}(x) = \arg \min_z \frac{1}{2t}\|z-x\|_2^2+h(z) $$

and the proximal gradient step is

$$x^{k+1} = \text{prox}_{th}(x^k-t\nabla g(x^k))$$